In [3]:
import json

# Define a function to load data from a file
def load_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Load the data from the three files
test_data_path = 'test_bio_labels.json'
train_data_path = 'train_bio_labels.json'
val_data_path = 'val_bio_labels.json'

test_data = load_data(test_data_path)
train_data = load_data(train_data_path)
val_data = load_data(val_data_path)
# Adjusting approach to properly display the data structure and a sample

# Function to safely extract samples from data which might be a dictionary or list
def get_sample(data):
    if isinstance(data, dict):
        # If the data is a dictionary, return a part of it
        return {k: data[k] for k in list(data)[:2]}
    elif isinstance(data, list):
        # If the data is a list, return the first two elements
        return data[:2]
    else:
        return "Unsupported data type"

# Get samples from each dataset
test_data_sample = get_sample(test_data)
train_data_sample = get_sample(train_data)
val_data_sample = get_sample(val_data)

{
    "test_data_sample": test_data_sample,
    "train_data_sample": train_data_sample,
    "val_data_sample": val_data_sample,
}

{'test_data_sample': {'03f3901e95ed493b866bd7807f623bc0': {'text': "True, our Constitution has no 'due process' clause or the VIII Amendment; but, in this branch of law, after R.C. Cooper v. Union of India, (1970) 1 SCC 248 and Maneka Gandhi v. Union of India, (1978) 1 SCC 248, the consequence is the same.",
   'labels': ['O',
    'O',
    'B_STATUTE',
    'I_STATUTE',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'B_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'B_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'O',
    'O',
    'O',
    'O',
    'O']},
  'b0311cba3aac4d909eec6e156c059617': {'text': 

In [4]:
import pandas as pd
# Convert dictionary data to DataFrame
train_df = pd.DataFrame.from_dict(train_data, orient='index').reset_index(drop=True)
test_df = pd.DataFrame.from_dict(test_data, orient='index').reset_index(drop=True)
val_df = pd.DataFrame.from_dict(val_data, orient='index').reset_index(drop=True)

# Display a sample from the train DataFrame to check preprocessing
train_df.head()

,text,labels
0,"Therefore, while interpreting statutory provis...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,The petitioner in W.P.No.15821 of 2008 was nev...,"[O, O, O, B_CASE_NUMBER, I_CASE_NUMBER, I_CASE..."
2,"The factum of accident, allegation of rash and...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B_OTHE..."
3,..36.. \n\n W.A.No.655/2012 & others Meaning t...,"[O, O, B_CASE_NUMBER, I_CASE_NUMBER, O, O, O, ..."
4,The law on this issue is well settled and the ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [6]:
import numpy as np

word_vectors = {}
glove_file = '/Users/arnav/Desktop/MachineLearning/CSE556-NLP-Assignments/Assignment 2/Embeddings/GloVe/glove.6B/glove.6B.100d.txt'

with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.split()
        word = parts[0]
        vector = np.array([float(val) for val in parts[1:]])
        word_vectors[word] = vector

# Load the GloVe embeddings
def get_glove_embedding(sentence, word_vectors, dim=100):
    words = sentence.split()
    embedding = np.zeros((len(words), dim))

    for i, word in enumerate(words):
        if word in word_vectors:
            embedding[i] = word_vectors[word]

    return embedding

In [7]:
# process and generate the embeddings for the first sentence in the train data
sentence = train_df['text'][0]
labels = train_df['labels'][0]
embedding = get_glove_embedding(sentence, word_vectors)
print(embedding.shape)
print(len(labels))

(21, 100)
21


In [8]:
def text_to_glove_embedding(text, embeddings, vector_size = 512, max_length=70):
    vectors = []
    words = text.split()
    for word in words:
        vector = embeddings.get(word)
        if vector is not None:
            vectors.append(vector)
        else:
            vectors.append(np.zeros(vector_size))
    
    # Pad the vectors with zeros to the max_length
    if len(vectors) < max_length:
        vectors += [np.zeros(vector_size)] * (max_length - len(vectors))
    else:
        vectors = vectors[:max_length]
    
    return np.array(vectors)
        

In [9]:
# Test the function with a sample text
text = "the cat sat on the mat"
glove_embedding = text_to_glove_embedding(text, glove_embeddings)
glove_embedding

NameError: name 'glove_embeddings' is not defined

In [ ]:
# converting all the sentences to glove embeddings
train_df['glove_embedding'] = train_df['text'].apply(lambda x: text_to_glove_embedding(x, glove_embeddings))
test_df['glove_embedding'] = test_df['text'].apply(lambda x: text_to_glove_embedding(x, glove_embeddings))
val_df['glove_embedding'] = val_df['text'].apply(lambda x: text_to_glove_embedding(x, glove_embeddings))

C:\Users\amilb\AppData\Local\Temp\ipykernel_16420\3127848815.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(vectors)
C:\Users\amilb\AppData\Local\Temp\ipykernel_16420\3127848815.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(vectors)
C:\Users\amilb\AppData\Local\Temp\ipykernel_16420\3127848815.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you

In [ ]:
print(train_df['glove_embedding'][0])

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 